In [1]:
import seaborn as sns
import  matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop,SGD,Adagrad,Adam
from tensorflow.nn import relu

C:\Users\dexhrestha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dexhrestha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dexhrestha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dexhrestha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

In [2]:
train_data = pd.read_csv('train_data/train.csv')
test_data = pd.read_csv('test_data/test.csv')

In [3]:
# remove rows with NaN
train_data.fillna(99999,inplace=True)
test_data.fillna(99999,inplace=True)
# train_data.dropna(inplace=True)
# test_data.dropna(inplace=True)
assert train_data.isna().sum().sum() <= 0  
assert test_data.isna().sum().sum() <= 0  

In [4]:
train_data.isna().sum().sum()

0

In [5]:
def preprocessing(data):
    data = data.copy()
    data.fillna(99999,inplace=True)
    # remove rows with NaN
    assert data.isna().sum().sum() <= 0  
    #drop unwanted features
    data.drop(columns=['User_ID','Product_ID','Marital_Status','Stay_In_Current_City_Years'],inplace=True)
    #convert categorical values to category dtype
    for column in data.columns[:-1]:
        data[column] = data[column].astype('category') 
    #one hot encoding
    data = pd.get_dummies(data,columns=['Age','City_Category'],prefix=['Age','City_Category'])
    #label encoding
    data['Gender'] = data['Gender'].cat.codes

    return data

In [6]:
train_data = preprocessing(train_data)
test_data = preprocessing(test_data)

In [7]:
#shift target variable to end 
cols = list(train_data.columns)
cols.remove('Purchase')
new_cols = cols + ['Purchase']
train_data = train_data[new_cols]

In [8]:
X = np.asarray(train_data.iloc[:,:-1])
Y = np.asarray(train_data.iloc[:,-1:])
# Y = [x[0] for x in Y]

In [9]:
X.shape

(550068, 15)

In [10]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

In [11]:
x_train.shape

(440054, 15)

In [16]:
def build_model():
  model = Sequential([
    Dense(64, activation=relu, input_shape=[x_train.shape[1]]),
    Dense(64, activation=relu),
    Dense(1)
  ])

  optimizer = RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [17]:
model = build_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [18]:
example_batch = x_train[:10]
example_result = model.predict(example_batch)
example_result

array([[1.3377690e+00],
       [1.6326005e+00],
       [3.0395285e+04],
       [3.0396145e+04],
       [8.0634902e+03],
       [3.0395785e+04],
       [3.0395172e+04],
       [3.0395934e+04],
       [2.9256444e+00],
       [3.0396105e+04]], dtype=float32)

In [23]:
from tensorflow.keras.callbacks import Callback

class PrintDot(Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  x_train[:10000], y_train[:10000],
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..........................................................................................

In [24]:
loss, mae, mse = model.evaluate(x_test, y_test, verbose=0)

In [25]:
mae

3319.1357

In [26]:
mse

19561014.0